In [ ]:
import itertools
import time
import multiprocessing
import sqlite3

def generate_percentages(current, remaining, k, min_percentage, max_percentage):
    if k == 1:
        if min_percentage <= remaining <= max_percentage:
            current_comb = sorted(current + [remaining])
            return [tuple(current_comb)]
        return []

    combinations = []
    for i in range(min_percentage, min(remaining - k + 1, max_percentage) + 1):
        combinations.extend(generate_percentages(current + [i], remaining - i, k - 1, min_percentage, max_percentage))
    return combinations

def count_combinations_for_element(element_comb, total_percentage, num_elements, min_percentage, max_percentage, print_interval):
    unique_combinations = set()
    count = 0

    percentage_combinations = generate_percentages([], total_percentage, num_elements, min_percentage, max_percentage)
    for percentage_comb in percentage_combinations:
        unique_combinations.add((tuple(sorted(element_comb)), percentage_comb))
        count += 1
        if count % print_interval == 0:
            print(f"Counted {count} valid combinations so far for element combination {element_comb}")

    return unique_combinations

def store_combinations_in_db(db_conn, combinations):
    cursor = db_conn.cursor()
    for element_comb, percentage_comb in combinations:
        cursor.execute("INSERT OR IGNORE INTO combinations (elements, percentages) VALUES (?, ?)",
                       (str(element_comb), str(percentage_comb)))
    db_conn.commit()

def create_db(db_name):
    db_conn = sqlite3.connect(db_name)
    cursor = db_conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS combinations (elements TEXT, percentages TEXT, UNIQUE(elements, percentages))")
    db_conn.commit()
    return db_conn

def count_combinations(elements, num_elements, total_percentage, min_percentage, max_percentage, db_name, print_interval=100000000, batch_size=16000):
    element_combinations = list(itertools.combinations(elements, num_elements))
    print(f"Number of element combinations: {len(element_combinations)}")

    num_batches = (len(element_combinations) + batch_size - 1) // batch_size  # Calculate the number of batches
    db_conn = create_db(db_name)

    for batch_idx in range(num_batches):
        print(f"Processing batch {batch_idx + 1} of {num_batches}")
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(element_combinations))
        batch_combinations = element_combinations[start_idx:end_idx]

        pool = multiprocessing.Pool(multiprocessing.cpu_count())  # Use all available CPU cores
        results = []

        for element_comb in batch_combinations:
            result = pool.apply_async(
                count_combinations_for_element,
                (element_comb, total_percentage, num_elements, min_percentage, max_percentage, print_interval)
            )
            results.append(result)

        pool.close()
        pool.join()

        for result in results:
            store_combinations_in_db(db_conn, result.get())

        print(f"Batch {batch_idx + 1} processed.")

    db_conn.close()

# Parameters
elements = list(range(1, 59))  # Elements labeled from 1 to 58
num_elements = 4
total_percentage = 100
min_percentage = 1
max_percentage = 97
db_name = 'combinations.db'

# Start the timer
start_time = time.time()

# Count valid combinations
count_combinations(elements, num_elements, total_percentage, min_percentage, max_percentage, db_name)
print("Completed counting combinations and storing them in the database.")

# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print(f"Time taken to execute the code: {elapsed_time:.6f} seconds")


In [ ]:
import sqlite3

def get_total_combinations(db_name):
    db_conn = sqlite3.connect(db_name)
    cursor = db_conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM combinations")
    total_combinations = cursor.fetchone()[0]
    db_conn.close()
    return total_combinations

# Database name
db_name = 'combinations.db'

# Get total unique combinations from the database
total_combinations = get_total_combinations(db_name)
print(f"Total unique combinations: {total_combinations}")
